In [2]:
from ngsolve import *
from ngsolve import Draw, curl, grad
from netgen.occ import *
from netgen.gui import *
from coil_geometry import *

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2405
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 14 thread(s)


2024-12-28 15:36:57.517 Python[29209:144157] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-28 15:36:57.517 Python[29209:144157] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [3]:
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilradius = 0.01  # Radius of the coil

# Geometrie laden
geo = create_homo_geometry(nwindings, wireradius, coilradius)

In [4]:
Draw(geo)

In [5]:
mesh = Mesh(
    geo.GenerateMesh()
)
mesh.Curve(1) 

 Face 1 / 10 (parameter space projection)
 Face 2 / 10 (parameter space projection)
 Face 3 / 10 (parameter space projection)
 Face 4 / 10 (parameter space projection)
 Face 5 / 10 (parameter space projection)
 Face 6 / 10 (parameter space projection)
 Face 7 / 10 (parameter space projection)
 Face 8 / 10 (parameter space projection)
 Face 9 / 10 (parameter space projection)
 Face 10 / 10 (parameter space projection)
 Delaunay meshing
 Remove Illegal Elements
 Delaunay meshing
 start tetmeshing
 Success !
 Volume Optimization
 Curve elements, order = 1


In [6]:
mesh.GetBoundaries()

('default',
 'default',
 'default',
 'default',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer')

In [7]:
# Konstanten
mu = 4 * pi * 1e-7  # Magnetische Feldkonstante (S.2)
sigma_coil = 5.87e7  # Leitfähigkeit Kupfer (S.11)
sigma_air = 0 # Leitfähigkeit Luft
k = 1e-5  # Regularisierung

# Geometrische Parameter
i_c = 500 
A_c = pi * wireradius**2  
r = sqrt(x**2 + y**2 + z**2) 

# Stromdichte j_c gemäß Aufgabenstellung
j_c = CoefficientFunction((
    (nwindings * i_c / A_c) * y / r,
    -(nwindings * i_c / A_c) * x / r,
    0
))

In [8]:
# H(curl)-Raum definieren
order = 2
V = HCurl(mesh, order=order-1, nograds=True, dirichlet="outer")
u, v = V.TnT()
gfA = GridFunction(V)


# Bilinearform und Linearform
a = BilinearForm(V, symmetric=True)
a += (1/mu) * InnerProduct(curl(u), curl(v)) * dx
a += k * InnerProduct(u, v) * dx

# Preconditioner
pre = Preconditioner(a, "multigrid")

# Linearform: Stromdichte
f = LinearForm(V)
f += InnerProduct(j_c, v) * dx

# Assemble
a.Assemble()
f.Assemble()

print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")
print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")

hcurl smoothingblocks, SmoothingType = 2
Matrix size: 47895 x 47895
Vector size: 47895
Präkonditionierer-Matrixgröße: 47895 x 47895


In [10]:
# Lösen
inv = CGSolver(a.mat, pre.mat, maxier=200)
with TaskManager():
    gfA.vec.data = inv * f.vec

# Visualisierung
Draw(curl(gfA), mesh, "MagneticField")
Draw(gfA, mesh, "VectorPotential")

TypeError: CGSolver(): incompatible function arguments. The following argument types are supported:
    1. (mat: BaseMatrix, pre: BaseMatrix, complex: bool = False, printrates: bool = True, precision: float = 1e-08, maxsteps: int = 200, conjugate: bool = False, maxiter: Optional[int] = None) -> ngsolve.la.KrylovSpaceSolver

Invoked with: <ngsolve.la.SparseMatrixd object at 0x117989c70>, <ngsolve.comp.MultiGridPreconditioner object at 0x11164b540>; kwargs: maxier=200